In [0]:
# load doc into memory
def load_doc(mylyric):
	# open the file as read only
	file = open(mylyric, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [0]:
# load document
in_filename = 'mylyric.txt'
doc = load_doc(in_filename)
print(doc[:200])

lyrics
"Verse One:

Alright I might
Have had a little glare when I stared at ya ho
But I didn't know she was like that
She stared right back
My niggas warnin me that she was comin on to me
I react lik


In [0]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [0]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['lyrics', 'verse', 'one', 'alright', 'i', 'might', 'have', 'had', 'a', 'little', 'glare', 'when', 'i', 'stared', 'at', 'ya', 'ho', 'but', 'i', 'didnt', 'know', 'she', 'was', 'like', 'that', 'she', 'stared', 'right', 'back', 'my', 'niggas', 'warnin', 'me', 'that', 'she', 'was', 'comin', 'on', 'to', 'me', 'i', 'react', 'like', 'a', 'mack', 'do', 'i', 'act', 'cool', 'just', 'to', 'test', 'her', 'cause', 'i', 'aint', 'no', 'jester', 'i', 'suggest', 'her', 'and', 'her', 'friend', 'be', 'outtie', 'cause', 'i', 'dont', 'want', 'to', 'make', 'my', 'pals', 'get', 'rowdy', 'and', 'doubt', 'me', 'our', 'friendship', 'but', 'when', 'lips', 'touch', 'i', 'go', 'crazy', 'in', 'the', 'clutch', 'sorta', 'like', 'schitzo', 'i', 'forgets', 'my', 'bros', 'and', 'pals', 'over', 'gals', 'i', 'didnt', 'mean', 'to', 'but', 'when', 'you', 'fiend', 'you', 'do', 'strange', 'things', 'for', 'the', 'denim', 'no', 'matter', 'whos', 'in', 'em', 'grab', 'a', 'flooze', 'then', 'im', 'in', 'traffic', 'dont', 'laugh',

In [0]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 241278


In [0]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [0]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [0]:
from keras.preprocessing.text import Tokenizer


In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [0]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [0]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [0]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            605900    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 12118)             1223918   
Total params: 1,980,718
Trainable params: 1,980,718
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=1)

Epoch 1/1
241278/241278 [==============================] - 739s 3ms/step - loss: 6.3696 - acc: 0.0460


In [0]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [0]:
seq_length = len(lines[0].split()) - 1

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

know if ill recuperate think its serious gone from bad to worse and im in trouble i think im coming down with something i know it gonna need your medicine so help me now im freaking out lover this love is serious everybody knows im mad for you you get me

i know i know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know you know
